In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Utilities as U

In [3]:
omp = 1
mpi = 4

# Tutorial for the YamboCalculator class

This tutorial describes the usage of the YamboCalculator class, that manages the run of (many) calculations in
parallel with the Yambo package.

## Perform an Hartree-Fock computation for Silicon

__Follow the tutorial for the QeCalculator to produce the .save folder needed to generate the yambo SAVE.__

The first step needed to perform a Yambo computation is to generate the SAVE folder from a QuantumESPRESSO
computation. 

The mppi.Utilities module has a function that perform this task

In [13]:
source_dir = 'QeCalculator_test/bands_8.save'
run_dir = 'YamboCalculator_test'

In [14]:
U.build_SAVE(source_dir,run_dir)

Create folder YamboCalculator_test
Executing command: cd QeCalculator_test/bands_8.save; p2y -a 2
Create a symlink of /home/marco/Applications/MPPI/sphinx_source/tutorials/QeCalculator_test/bands_8.save/SAVE in YamboCalculator_test
Executing command: cd YamboCalculator_test;OMP_NUM_THREADS=1 yambo


Now the YamboInput class can create the yambo input object.

We consider a HF computation

In [24]:
exx = 3.0 # Hartree
inp = I.YamboInput(args='yambo -x -V rl',folder=run_dir)
inp.set_kRange(1,2) #restrict the analysis to the first two kpoints
inp['variables']['EXXRLvcs'] = [exx*1e3,'mHa']
name = 'hf_exx'+str(exx)
jobname = 'hf_job_exx'+str(exx)
inp

{'args': 'yambo -x -V rl',
 'folder': 'YamboCalculator_test',
 'filename': 'yambo.in',
 'arguments': ['HF_and_locXC'],
 'variables': {'FFTGvecs': [2133.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [3000.0, 'mHa'],
  'VXCRLvcs': [13107.0, 'RL'],
  'QPkrange': [[1, 2, 1, 8], '']}}

To run the computation we create an istance of YamboCalculator. This object behaves almost exactly as
the QeCalculator for what concerns the user interface

In [16]:
C.YamboCalculator?

Init signature:
C.YamboCalculator(
    omp=1,
    mpi=2,
    mpi_run='mpirun -np',
    executable='yambo',
    scheduler='direct',
    skip=True,
    clean_restart=True,
    verbose=True,
    **kwargs,
)
Docstring:     
Perform a Yambo calculation. Computations are managed by a scheduler that,
in the actual implementation of the class, can be `direct` or `slurm`.

Parameters:
   omp (:py:class:`int`) : value of the OMP_NUM_THREADS variable
   mpi (:py:class:`int`) : number of mpi processes
   mpi_run (:py:class:`string`) : command for the execution of mpirun, e.g. 'mpirun -np' or 'mpiexec -np'
   executable (:py:class:`string`) : set the executable (yambo, ypp, yambo_rt, ...) of the Yambo package
   scheduler (:py:class:`string`) : choose the scheduler used to submit the job, actually the choices implemented are
        'direct' that runs the computation using the python subprocess package and 'slurm' that creates a slurm script
   skip (:py:class:`bool`) : if True evaluate if the comp

In [25]:
code = C.YamboCalculator(omp=omp,mpi=mpi)
code.global_options()

Initialize a Yambo calculator with scheduler direct


{'omp': 1,
 'mpi': 4,
 'mpi_run': 'mpirun -np',
 'executable': 'yambo',
 'scheduler': 'direct',
 'skip': True,
 'clean_restart': True,
 'verbose': True}

The structure of the folder in which yambo writes its results is governed by the name and jobname 
variables. It is possible to provide only the name variable. 

The effect of this choice can be seen in the command string executed by the calculator

In [29]:
result = code.run(run_dir=run_dir,input=inp,name=name,skip=False)
result

delete folder: YamboCalculator_test/hf_exx3.0
run command: cd YamboCalculator_test; mpirun -np 4 yambo -F hf_exx3.0.in -J hf_exx3.0 -C hf_exx3.0
computation hf_exx3.0 is running...
computation hf_exx3.0 ended


{'output': ['YamboCalculator_test/hf_exx3.0/o-hf_exx3.0.hf'],
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'HF_and_locXC': 'YamboCalculator_test/hf_exx3.0/ndb.HF_and_locXC'}

In this case yambo create the hf_exx3.0 folder and write all the files in these folders. Result is a dictionary that contain the names
of the o- file and the databases created by yambo.

Instead, if we set also set a jobname we have

In [28]:
result = code.run(run_dir=run_dir,input=inp,name=name,jobname=jobname,skip=False)
result

delete folder: YamboCalculator_test/hf_exx3.0
delete folder: YamboCalculator_test/hf_job_exx3.0
run command: cd YamboCalculator_test; mpirun -np 4 yambo -F hf_exx3.0.in -J hf_job_exx3.0 -C hf_exx3.0
computation hf_exx3.0 is running...
computation hf_exx3.0 ended


{'output': ['YamboCalculator_test/hf_exx3.0/o-hf_job_exx3.0.hf'],
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'HF_and_locXC': 'YamboCalculator_test/hf_job_exx3.0/ndb.HF_and_locXC'}

In this case yambo creates also the folders hf_job_exx and write the databases in these folders.

### Usage of the skip parameter

If we repeat a calculation that has been already performed and skip = True the class skip its computation, for instance

In [23]:
result = code.run(run_dir = run_dir, input = inp, name= name, skip = True)
result

Skip the run of hf_exx3.0


{'output': ['YamboCalculator_test/hf_exx3.0/o-hf_exx3.0.hf'],
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'HF_and_locXC': 'YamboCalculator_test/hf_exx3.0/ndb.HF_and_locXC'}

### Test of the slurm scheduler

If the `slurm` scheduler is chosen the calculator prepare the slurm script and submit it. The effects of skip and
multiTask parameters can be tested

In [ ]:
results = code.run(run_dir=run_dir,inputs=inputs,names=names, scheduler = 'slurm', skip = False, multiTask = True)
results

The slurm script is written in the run_dir. The execution of the run requires that the slurm scheduler is installed.

## Perform a GW computation for Silicon

We make usage of the YamboCalculator to perform a different yambo computation. In this way we control
how this class manage the output files and the ndb database in various cases.

In [30]:
inp = I.YamboInput(args='yambo -d -k hartee -g n -p p -V qp',folder=run_dir)
inp.set_kRange(1,2)
#inp

In [31]:
result = code.run(input = inp, run_dir = run_dir, name='qp_test1')
result

run command: cd YamboCalculator_test; mpirun -np 4 yambo -F qp_test1.in -J qp_test1 -C qp_test1
computation qp_test1 is running...
computation qp_test1 ended


{'output': ['YamboCalculator_test/qp_test1/o-qp_test1.qp'],
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'pp': 'YamboCalculator_test/qp_test1/ndb.pp',
 'QP': 'YamboCalculator_test/qp_test1/ndb.QP',
 'HF_and_locXC': 'YamboCalculator_test/qp_test1/ndb.HF_and_locXC',
 'dipoles': 'YamboCalculator_test/qp_test1/ndb.dipoles'}

Perform the same computation but specify also a jobname 

In [18]:
result = code.run(input = inp, run_dir = run_dir, name='qp_test2', jobname = 'qp_job_test2')
result

Skip the run of qp_test2


{'output': ['YamboCalculator_test/qp_test2/o-qp_job_test2.qp'],
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'pp': 'YamboCalculator_test/qp_job_test2/ndb.pp',
 'QP': 'YamboCalculator_test/qp_job_test2/ndb.QP',
 'HF_and_locXC': 'YamboCalculator_test/qp_job_test2/ndb.HF_and_locXC',
 'dipoles': 'YamboCalculator_test/qp_job_test2/ndb.dipoles'}

## Test of the ExtendOut option in the input file

The ExtendOut option enables the writing of all the variables in the output files of Yambo. 
This feature has no effect for an HF computation and we test for a QP one:

In [26]:
inp = I.YamboInput(args='yambo -d -k hartee -g n -p p -V qp',folder=run_dir)
inp.set_kRange(1,2)
inp.set_extendOut()
#inp

In [27]:
result = code.run(input = inp, run_dir = run_dir,name='qp_test_ExtendOut')
result

Skip the run of qp_test_ExtendOut


{'output': ['YamboCalculator_test/qp_test_ExtendOut/o-qp_test_ExtendOut.qp'],
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'pp': 'YamboCalculator_test/qp_test_ExtendOut/ndb.pp',
 'QP': 'YamboCalculator_test/qp_test_ExtendOut/ndb.QP',
 'HF_and_locXC': 'YamboCalculator_test/qp_test_ExtendOut/ndb.HF_and_locXC',
 'dipoles': 'YamboCalculator_test/qp_test_ExtendOut/ndb.dipoles'}

you can check that the o- files contain more information. These feature is managed by the YamboParser class of the package.

## Perform a ypp computation

The YamboCalculator class can manage also ypp computation.

Let's see an example by performing a band calculation along a path

In [28]:
inp = I.YamboInput(args='ypp -s b',folder=run_dir,filename='ypp.in') 
inp

{'args': 'ypp -s b',
 'folder': 'YamboCalculator_test',
 'filename': 'ypp.in',
 'arguments': [],
 'variables': {'OutputAlat': [0.0, ''],
  'INTERP_Shell_Fac': [20.0, ''],
  'INTERP_NofNN': [1.0, ''],
  'BANDS_steps': [10.0, ''],
  'INTERP_mode': 'NN',
  'cooIn': 'rlu',
  'cooOut': 'rlu',
  'CIRCUIT_E_DB_path': 'none',
  'PROJECT_mode': 'none',
  'BANDS_bands': [[1, 8], '']}}

We define a calculator for ypp. This calculation requires 1 mpirun (see yambo for further information)

In [29]:
code = C.YamboCalculator(mpi=1,executable='ypp',skip=False)
code.global_options()

Initialize a Yambo calculator with scheduler direct


{'omp': '1',
 'mpi': 1,
 'mpi_run': 'mpirun -np',
 'executable': 'ypp',
 'scheduler': 'direct',
 'skip': False,
 'clean_restart': True,
 'verbose': True}

Set the input parameter to perform the band computation along a path

In [30]:
# in alat
G = [0.,0.,0.]
X = [1.,0.,0.]
L = [0.5,0.5,0.5]
K = [1.0,0.5,0.]
path = [L,G,X,K,G]

band_range = [2,5]
bands_step = 30

In [31]:
# scissor
# inp['variables']['GfnQP_E'] = [1.0,1.0,1.0]

# band structure
# Some methods that perform these operation can be added in the YamboInput class
inp['variables']['BANDS_steps'] = [bands_step,'']
inp['variables']['BANDS_bands'] = [band_range,'']
inp['variables']['BANDS_kpts'] = [path,'']
inp['variables']['cooIn'] = 'alat'
inp['variables']['cooOut'] = 'alat'
inp

{'args': 'ypp -s b',
 'folder': 'YamboCalculator_test',
 'filename': 'ypp.in',
 'arguments': [],
 'variables': {'OutputAlat': [0.0, ''],
  'INTERP_Shell_Fac': [20.0, ''],
  'INTERP_NofNN': [1.0, ''],
  'BANDS_steps': [30, ''],
  'INTERP_mode': 'NN',
  'cooIn': 'alat',
  'cooOut': 'alat',
  'CIRCUIT_E_DB_path': 'none',
  'PROJECT_mode': 'none',
  'BANDS_bands': [[2, 5], ''],
  'BANDS_kpts': [[[0.5, 0.5, 0.5],
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [1.0, 0.5, 0.0],
    [0.0, 0.0, 0.0]],
   '']}}

In [32]:
result = code.run(run_dir=run_dir,input=inp,name='bands_test1')
result

delete folder: YamboCalculator_test/bands_test1
run command: cd YamboCalculator_test; mpirun -np 1 ypp -F bands_test1.in -J bands_test1 -C bands_test1
computation bands_test1 is running...
computation bands_test1 ended


{'output': ['YamboCalculator_test/bands_test1/o-bands_test1.bands_interpolated'],
 'dft': 'YamboCalculator_test/SAVE/ns.db1'}

Result can be parsed using the YamboParser class of this package or other external tools

Test the jobname specification for this type of computations

In [33]:
result = code.run(run_dir=run_dir,input=inp,name='bands_test2',jobname='bands_job_test2')
result

delete folder: YamboCalculator_test/bands_test2
run command: cd YamboCalculator_test; mpirun -np 1 ypp -F bands_test2.in -J bands_job_test2 -C bands_test2
computation bands_test2 is running...
computation bands_test2 ended


{'output': ['YamboCalculator_test/bands_test2/o-bands_job_test2.bands_interpolated'],
 'dft': 'YamboCalculator_test/SAVE/ns.db1'}